In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Ridge

In [2]:
# Set parameters for linear regression

# ridge parameters for random search
fit_intercept = [True, False]
copy_X = [True, False]
alpha = [0.1, 0.5, 1, 2, 5, 10, 20, 50, 100]
tol = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']

# Create the random grid
random_grid = {'fit_intercept': fit_intercept,
                'copy_X': copy_X,
                'alpha': alpha,
                'tol': tol,
                'solver': solver}


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_032022_032023_taxi_rides.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})

In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# Define the feature columns and the target column
feature_cols = ['time']
target_col = 'y'

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = Ridge()

search = RandomizedSearchCV(estimator = model, 
                               param_distributions = random_grid, 
                               n_iter = 25, 
                               cv = tscv,
                               refit=True, 
                               verbose=3, 
                               random_state=42, 
                               n_jobs = -1, 
                               scoring=scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                               error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[CV 2/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-1.666 total time=   0.0s
[CV 5/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-1.260 total time=   0.0s
[CV 4/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-1.300 total time=   0.0s
[CV 2/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-1.666 total time=   0.0s
[CV 1/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-1.425 total time=   0.0s
[CV 5/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-1.260 total time=   0.0s
[CV 4/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-1.300 total time=   0.0s
[CV 1/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-1.425 total time=   0.0s
[CV 1/5] END alpha=100, copy_X=False, fit_intercept=Fals

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Ridge(), n_iter=25, n_jobs=-1,
                   param_distributions={'alpha': [0.1, 0.5, 1, 2, 5, 10, 20, 50,
                                                  100],
                                        'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'solver': ['auto', 'svd', 'cholesky',
                                                   'lsqr', 'sparse_cg', 'sag',
                                                   'saga'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1, 1, 10,
                                                100]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -1.399620700067074
Best Hyperparameters: {'tol': 100, 'solver': 'sparse_cg', 'fit_intercept': True, 'copy_X': False, 'alpha': 10}
Best Model: Ridge(alpha=10, copy_X=False, solver='sparse_cg', tol=100)
Best Index: 24
CV Results: {'mean_fit_time': array([0.00331774, 0.00577774, 0.00340743, 0.00257349, 0.00222802,
       0.00161295, 0.00905762, 0.00189919, 0.00165944, 0.00148935,
       0.00177321, 0.00159116, 0.00145903, 0.00141721, 0.00153379,
       0.00376778, 0.00142097, 0.00129585, 0.00118685, 0.00118904,
       0.00134988, 0.00187759, 0.00152001, 0.00144362, 0.00181084]), 'std_fit_time': array([2.09406521e-03, 2.80858199e-03, 1.57450224e-03, 1.10657316e-03,
       4.45690307e-04, 8.25147930e-05, 4.59191889e-03, 3.41929130e-04,
       2.17373811e-04, 1.22993062e-04, 5.59988870e-04, 1.84000666e-04,
       2.09558110e-04, 2.52361551e-04, 2.40140060e-04, 1.66237079e-03,
       4.48986739e-04, 3.34000362e-04, 1.77733335e-04, 1.55162943e-04,
       2.32696113e-04, 6.49589249e